In [89]:
import pandas as pd
import numpy as np
import requests
#python client for Socrata open data API
from sodapy import Socrata
import os
import seaborn as sns
from matplotlib import pyplot as plt
import sys

os.getcwd()
pd.options.display.max_columns=50

In [19]:
#define parameters for endpoint, dataset, and app token
data_url = 'data.cityofnewyork.us'
dataset = 'erm2-nwe9'
app_token = 'dM7DDeidbAmgtydtJVV1epbiU'

#sets up the connection, need application token to override throttling limits
#username and password only required for creating or modifying data
client = Socrata(data_url, app_token)
client.timeout = 6000

#count number of records in desired dataset
record_count = client.get(dataset, select='count(*)', where="created_date >='2019-01-01'")
record_count

[{'count': '3049868'}]

In [20]:
def get_data(chunk_size=100000, total_rows=300000):
    start = 0
    results=[]

    #paginate through dataset in sets of 10000 to get all records since 2019
    while True:
        print(start)
        results.extend(client.get(dataset,where="created_date >= '2019-01-01'", 
                                  limit=chunk_size, offset=start))
        start += chunk_size
        if start > total_rows:
            break

0
100000
200000


In [205]:
def reduce_mem_usage(dataframe):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = dataframe.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in dataframe.columns:
        col_type = dataframe[col].dtype
        
        if col_type != object:
            c_min = dataframe[col].min()
            c_max = dataframe[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    #print(col, col_type, 'int8')
                    dataframe[col] = dataframe[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    #print(col, col_type, 'int16')
                    dataframe[col] = dataframe[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    #print(col, col_type, 'int32')
                    dataframe[col] = dataframe[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    #print(col, col_type, 'int64')
                    dataframe[col] = dataframe[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    #print(col, col_type, 'float16')
                    dataframe[col] = dataframe[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    #print(col, col_type, 'float32')
                    dataframe[col] = dataframe[col].astype(np.float32)
                else:
                    #print(col, col_type, 'float64')
                    dataframe[col] = dataframe[col].astype(np.float64)
        else:
            #print(col, col_type, 'category')
            dataframe[col] = dataframe[col].astype('category')
            
    end_mem = dataframe.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return dataframe


def get_mem_usage(df):
    dtypes = df.dtypes.astype(str).unique()
    total_mem = df.memory_usage().sum()/1024**2
    for dtype in dtypes:
        selected_dtype = df.select_dtypes(include=dtype)
        mean_usage_b = selected_dtype.memory_usage().mean()
        mean_usage_mb = mean_usage_b / 1024**2
        print('avg memory usage for {} columns: {:03.2f} MB'.format(dtype, mean_usage_mb))
    print(f'total memory usage: {total_mem:.2f}')

In [209]:
#orig_results - all dtypes are objects
#orig_results = pd.DataFrame(results)
#orig_results.to_csv('/Users/christinejiang/Documents/Python/data/311_data.csv', index=False)

#olddf - most dtypes are objects, some are ints and some are floats
#BUT olddf datatypes get changed inplace after running reduce_mem_usage! becomes mostly categorical
olddf = pd.read_csv('/Users/christinejiang/Documents/Python/data/311_data.csv')

#newdf - most dtypes are categorical, some ints and floats
#newdf = reduce_mem_usage(olddf)

/Users/christinejiang/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (8,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [206]:
#calculate average mem usage per datatype
#in order from highest to lowest memory: object, category, float32, float16, int32
print('orig results')
get_mem_usage(orig_results)

print('olddf')
get_mem_usage(olddf)

print('newdf')
get_mem_usage(newdf)

orig results
avg memory usage for object columns: 2.23 MB
total memory usage: 93.84
olddf
avg memory usage for int32 columns: 0.57 MB
avg memory usage for category columns: 1.65 MB
avg memory usage for float32 columns: 0.86 MB
avg memory usage for float16 columns: 0.38 MB
total memory usage: 64.94
newdf
avg memory usage for int32 columns: 0.57 MB
avg memory usage for category columns: 1.65 MB
avg memory usage for float32 columns: 0.86 MB
avg memory usage for float16 columns: 0.38 MB
total memory usage: 64.94


In [441]:
#for each column, get the 
def reduce_df_memory(df):
    orig_size = df.memory_usage().sum()/1024**2
    dtypes = df.dtypes.astype(str).unique()
    for dtype in dtypes:
        if 'float' in dtype:
            selected_float = df.select_dtypes(include='float')
            converted_float = selected_float.apply(pd.to_numeric, downcast='float')
            float_size = selected_float.memory_usage().sum()/1024**2
            converted_float_size = converted_float.memory_usage().sum()/1024**2
            print(f'floats: {float_size:.2f} reduced to {converted_float_size:.2f} MB')
        if 'int' in dtype:
            selected_int = df.select_dtypes(include='integer')
            converted_int = selected_int.apply(pd.to_numeric, downcast='integer')
            int_size = selected_int.memory_usage().sum()/1024**2
            converted_int_size = converted_int.memory_usage().sum()/1024**2
            print(f'ints: {int_size:.2f} reduced to {converted_int_size:.2f} MB')
        if 'object' in dtype:
            selected_object = df.select_dtypes(include='object')
            obj_size = selected_object.memory_usage().sum()/1024**2
            converted_obj = pd.DataFrame()
            for col in selected_object.columns:
                count = len(selected_object[col])
                unique = len(selected_object[col].unique())
                if unique < count/2:
                    converted_obj[col] = selected_object[col].astype('category')
                else:
                    converted_obj[col] = selected_object[col]
            converted_obj_size = converted_obj.memory_usage().sum()/1024**2
            print(f'object: {obj_size:.2f} reduced to {converted_obj_size:.2f} MB')

    float_int = converted_float.join(converted_int)
    reduced_df = float_int.join(converted_obj)
    reduced_df = reduced_df[df.columns]
    reduced_size = reduced_df.memory_usage().sum()/1024**2
    print(f'final df: {orig_size:.2f} reduced to {reduced_size:.2f} MB, {(orig_size-reduced_size)/orig_size*100:.1f}% reduction')
    return reduced_df

In [442]:
clean_olddf = reduce_df_memory(olddf)

ints: 2.29 reduced to 1.14 MB
object: 80.11 reduced to 48.73 MB
floats: 11.44 reduced to 5.72 MB
final df: 93.84 reduced to 55.59 MB, 40.8% reduction


In [448]:
olddf.select_dtypes(include='object').columns

Index(['created_date', 'closed_date', 'agency', 'agency_name',
       'complaint_type', 'descriptor', 'location_type', 'incident_zip',
       'incident_address', 'street_name', 'address_type', 'city',
       'facility_type', 'status', 'due_date', 'resolution_description',
       'resolution_action_updated_date', 'community_board', 'borough',
       'open_data_channel_type', 'park_facility_name', 'park_borough',
       'location', 'cross_street_1', 'cross_street_2', 'intersection_street_1',
       'intersection_street_2', 'taxi_company_borough',
       'taxi_pick_up_location', 'bridge_highway_name',
       'bridge_highway_direction', 'road_ramp', 'bridge_highway_segment',
       'landmark', 'vehicle_type'],
      dtype='object')

In [449]:
orig_results.select_dtypes(include='object').columns

Index(['unique_key', 'created_date', 'closed_date', 'agency', 'agency_name',
       'complaint_type', 'descriptor', 'location_type', 'incident_zip',
       'incident_address', 'street_name', 'address_type', 'city',
       'facility_type', 'status', 'due_date', 'resolution_description',
       'resolution_action_updated_date', 'community_board', 'borough',
       'x_coordinate_state_plane', 'y_coordinate_state_plane',
       'open_data_channel_type', 'park_facility_name', 'park_borough',
       'latitude', 'longitude', 'location', 'cross_street_1', 'cross_street_2',
       'bbl', 'intersection_street_1', 'intersection_street_2',
       'taxi_company_borough', 'taxi_pick_up_location', 'bridge_highway_name',
       'bridge_highway_direction', 'road_ramp', 'bridge_highway_segment',
       'landmark', 'vehicle_type'],
      dtype='object')

In [467]:
len(orig_results['cross_street_1'].unique())

9679

In [468]:
for col in orig_results.columns:
    print(col)
    test1 = len(orig_results[col].unique())

unique_key
created_date
closed_date
agency
agency_name
complaint_type
descriptor
location_type
incident_zip
incident_address
street_name
address_type
city
facility_type
status
due_date
resolution_description
resolution_action_updated_date
community_board
borough
x_coordinate_state_plane
y_coordinate_state_plane
open_data_channel_type
park_facility_name
park_borough
latitude
longitude
location


TypeError: unhashable type: 'dict'

In [455]:
orig_results.dtypes

unique_key                        object
created_date                      object
closed_date                       object
agency                            object
agency_name                       object
complaint_type                    object
descriptor                        object
location_type                     object
incident_zip                      object
incident_address                  object
street_name                       object
address_type                      object
city                              object
facility_type                     object
status                            object
due_date                          object
resolution_description            object
resolution_action_updated_date    object
community_board                   object
borough                           object
x_coordinate_state_plane          object
y_coordinate_state_plane          object
open_data_channel_type            object
park_facility_name                object
park_borough    